In [ ]:
# Start twitter cleaning / analysis

In [13]:
import pandas as pd
import numpy as np
import re
import json
import ast
import tweepy
from textblob import TextBlob
import os
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
import emoji
import yake

twitter_tweets = pd.read_json("../data/ore_tweets_content.json")
twitter_tweets

pd.set_option('max_columns', None)

In [3]:
#splitting the above dataframe

twitter_tweets_split = twitter_tweets['tweet'].apply(pd.Series)
twitter_tweets_split

,retweeted,source,entities,display_text_range,favorite_count,in_reply_to_status_id_str,id_str,in_reply_to_user_id,truncated,retweet_count,id,in_reply_to_status_id,created_at,favorited,full_text,lang,in_reply_to_screen_name,in_reply_to_user_id_str,possibly_sensitive,extended_entities
0,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 50]",0,1031515975928213505,1031526032153735168,29176998,False,0,1031526032153735168,1031515975928213505,Mon Aug 20 12:59:05 +0000 2018,False,@silentworks @jakuuire @pharingee Nice! I’m ga...,en,silentworks,29176998,NaN,NaN
1,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 16]",0,1031500040408387584,1031503428672008193,29176998,False,0,1031503428672008193,1031500040408387584,Mon Aug 20 11:29:16 +0000 2018,False,@silentworks 💀💀💀,und,silentworks,29176998,NaN,NaN
2,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 140]",0,NaN,1031263237025812485,NaN,False,0,1031263237025812485,NaN,Sun Aug 19 19:34:50 +0000 2018,False,RT @IbukunOg: Benjamin @DadaBen_ ! I’m so prou...,en,NaN,NaN,NaN,NaN
3,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 85]",0,NaN,1031262871240486912,NaN,False,0,1031262871240486912,NaN,Sun Aug 19 19:33:23 +0000 2018,False,RT @Mz_Chi: Here is a link to the video if you...,en,NaN,NaN,False,NaN
4,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 121]",0,NaN,1031262859899154432,NaN,False,0,1031262859899154432,NaN,Sun Aug 19 19:33:20 +0000 2018,False,RT @Mz_Chi: I'm going to make a YouTube video ...,en,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 140]",0,NaN,1236903007364632578,NaN,False,0,1236903007364632578,NaN,Mon Mar 09 06:33:52 +0000 2020,False,RT @DavisVilums: Exactly 5 years ago I started...,en,NaN,NaN,NaN,NaN
2700,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 97]",0,NaN,1236902584012509184,NaN,False,0,1236902584012509184,NaN,Mon Mar 09 06:32:11 +0000 2020,False,RT @paulbatum: Come work with me on Azure Func...,en,NaN,NaN,False,NaN
2701,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [{'text': 'dataviz', 'indices': [...","[0, 140]",0,NaN,1235819868873605120,NaN,False,0,1235819868873605120,NaN,Fri Mar 06 06:49:51 +0000 2020,False,RT @sxywu: ICYMI MY LATEST🎊is a physical #data...,en,NaN,NaN,NaN,NaN
2702,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 59]",2,1235524406778032128,1235568746711756801,29176998,False,1,1235568746711756801,1235524406778032128,Thu Mar 05 14:11:59 +0000 2020,False,@silentworks https://t.co/mBQ2Zx0Rqt\n\nYou kn...,en,silentworks,29176998,False,NaN


In [4]:
#splitting different user's mentioned into the same column
def get_user_mentions(entities):
    if entities is None:
        return []
    
    user_mentions_list = entities["user_mentions"]

    return [ single_mention["screen_name"] for single_mention in user_mentions_list]

twitter_tweets_split["users_mentioned"] = twitter_tweets_split["entities"].apply(get_user_mentions)

twitter_tweets_split["users_mentioned"]

0       [silentworks, jakuuire, pharingee]
1                            [silentworks]
2                     [IbukunOg, DadaBen_]
3                                 [Mz_Chi]
4                                 [Mz_Chi]
                       ...                
2699                         [DavisVilums]
2700                           [paulbatum]
2701                               [sxywu]
2702                         [silentworks]
2703                         [emeka_boris]
Name: users_mentioned, Length: 2704, dtype: object

In [5]:
#final tweets dataframe
twitter_tweets_final = twitter_tweets_split
twitter_tweets_final.columns

Index(['retweeted', 'source', 'entities', 'display_text_range',
       'favorite_count', 'in_reply_to_status_id_str', 'id_str',
       'in_reply_to_user_id', 'truncated', 'retweet_count', 'id',
       'in_reply_to_status_id', 'created_at', 'favorited', 'full_text', 'lang',
       'in_reply_to_screen_name', 'in_reply_to_user_id_str',
       'possibly_sensitive', 'extended_entities', 'users_mentioned'],
      dtype='object')

Here I will be performin sentiment analysis on the tweets. To do so I will first need to clean the contents of the tweets - specifically get rid of any text that says "RT" or user names.

In [7]:
#Creating new dataframe and new features
twitter_tweets_sentiment = pd.DataFrame(twitter_tweets_final)
twitter_tweets_sentiment["clean_text"] = twitter_tweets_sentiment["full_text"]
twitter_tweets_sentiment

#Identifying tweets that are retweets
for index in range(len(twitter_tweets_sentiment)):
    if bool("RT" in twitter_tweets_sentiment["clean_text"][index]) == True:
        twitter_tweets_sentiment["retweeted"][index] = True
    else:
        twitter_tweets_sentiment["retweeted"][index] = False
        

c:\users\oreog\miniconda3\envs\fusion\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
c:\users\oreog\miniconda3\envs\fusion\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [10]:
twitter_tweets_sentiment["retweeted"].value_counts()

#Removing RT, Punctuation etc
#create seperate column for RT

twitter_tweets_sentiment["clean_text"]= twitter_tweets_sentiment["clean_text"].replace(to_replace ='[RT]', value = '', regex = True)
twitter_tweets_sentiment["clean_text"]= twitter_tweets_sentiment["clean_text"].replace(to_replace ='@[\w:]+', value = '', regex = True)
twitter_tweets_sentiment["clean_text"] = twitter_tweets_sentiment.clean_text.str.lower()
twitter_tweets_sentiment["clean_text"][0:20]

#remove emojis
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

twitter_tweets_sentiment["clean_text"] = twitter_tweets_sentiment["clean_text"].apply(remove_emoji)

#remove links
links = "((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)"
twitter_tweets_sentiment["clean_text"]= twitter_tweets_sentiment["clean_text"].replace(to_replace =links, value = '', regex = True)
twitter_tweets_sentiment["clean_text"][156]

#remove punctuation
twitter_tweets_sentiment["clean_text"]= twitter_tweets_sentiment["clean_text"].replace(to_replace =r'[^\w\s]', value = '', regex = True)


#remove double white spaces
twitter_tweets_sentiment["clean_text"]= twitter_tweets_sentiment["clean_text"].replace(to_replace =r'[\s]{2,}', value = ' ', regex = True)

#remove white space from the start of sentences

twitter_tweets_sentiment["clean_text"] = twitter_tweets_sentiment["clean_text"].apply(lambda x: x.strip())
twitter_tweets_sentiment["clean_text"]

twitter_tweets_sentiment["full_text"][13]

'RT @Babajiide: This Would be good for @codebeast @segunfamisa @Soloxpress 😂 and we can put @unicodeveloper on watch list as he as a babe no…'

In [11]:
#Calculating sentiment using TextBlob. Polarity in this case is meant to show how positive or negative the tweet was, with super positive being 1 and negative being -1.

twitter_tweets_sentiment[['polarity', 'subjectivity']] = twitter_tweets_sentiment["clean_text"].apply(lambda text: pd.Series(TextBlob(text).sentiment))
twitter_tweets_sentiment[0:10]

#Calculating Negative, Positive, Neutral and Compound values using Vader
import nltk
nltk.download('vader_lexicon')

for index, row in twitter_tweets_sentiment["clean_text"].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score["neg"]
    neu = score["neu"]
    pos = score["pos"]
    comp = score["compound"]
    if score["neg"] > score["pos"]:
        twitter_tweets_sentiment.loc[index, "sentiment"] = 'negative'
    elif score["pos"] > score["neg"]:
        twitter_tweets_sentiment.loc[index, "sentiment"] = 'positive'
    else:
        twitter_tweets_sentiment.loc[index, "sentiment"] = 'neutral'
    twitter_tweets_sentiment.loc[index, "compound"] = score["compound"]


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\oreog\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,retweeted,source,entities,display_text_range,favorite_count,in_reply_to_status_id_str,id_str,in_reply_to_user_id,truncated,retweet_count,...,in_reply_to_screen_name,in_reply_to_user_id_str,possibly_sensitive,extended_entities,users_mentioned,clean_text,polarity,subjectivity,sentiment,compound
0,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 50]",0,1031515975928213505,1031526032153735168,29176998,False,0,...,silentworks,29176998,NaN,NaN,"[silentworks, jakuuire, pharingee]",nice im game,0.100000,0.700000,positive,0.4215
1,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 16]",0,1031500040408387584,1031503428672008193,29176998,False,0,...,silentworks,29176998,NaN,NaN,[silentworks],,0.000000,0.000000,neutral,0.0000
2,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 140]",0,NaN,1031263237025812485,NaN,False,0,...,NaN,NaN,NaN,NaN,"[IbukunOg, DadaBen_]",benjamin im so proud of you i see this guy tak...,0.245455,0.684848,positive,0.6113
3,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 85]",0,NaN,1031262871240486912,NaN,False,0,...,NaN,NaN,False,NaN,[Mz_Chi],here is a link to the video if you be interested,0.250000,0.500000,positive,0.4019
4,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 121]",0,NaN,1031262859899154432,NaN,False,0,...,NaN,NaN,NaN,NaN,[Mz_Chi],im going to make a youube video showing how yo...,0.000000,0.000000,negative,-0.1280
5,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'user_mentions': [{'name': 'The Chi', 'screen...","[0, 55]",0,NaN,1031262697109835777,NaN,False,0,...,NaN,NaN,False,{'media': [{'expanded_url': 'https://twitter.c...,[Mz_Chi],from dusk till dawn,0.000000,0.000000,neutral,0.0000
6,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 115]",0,NaN,1031205963964067842,NaN,False,0,...,NaN,NaN,False,NaN,[allenakinkunle],maybe you dont understand\nẹgbàmí what was he ...,0.000000,0.000000,neutral,0.0000
7,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 139]",0,NaN,1031147154709069824,NaN,False,0,...,NaN,NaN,NaN,NaN,[asemota],here is something that i discovered quite earl...,0.152083,0.360417,positive,0.8176
8,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [{'text': 'WISPDEFCONSQUAD', 'ind...","[0, 120]",0,NaN,1031074729610895360,NaN,False,0,...,NaN,NaN,False,NaN,[Ebunsky],woohoo my first linkedin article sharing my de...,0.250000,0.333333,positive,0.7269
9,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [{'text': 'forLoopPod', 'indices'...","[0, 140]",0,NaN,1030796350580031489,NaN,False,0,...,NaN,NaN,NaN,NaN,[forloopAfrica],update forlooppod is now available on iunes no...,0.300000,0.400000,positive,0.2263


In [14]:
twitter_tweets_sentiment.head(10)

,retweeted,source,entities,display_text_range,favorite_count,in_reply_to_status_id_str,id_str,in_reply_to_user_id,truncated,retweet_count,id,in_reply_to_status_id,created_at,favorited,full_text,lang,in_reply_to_screen_name,in_reply_to_user_id_str,possibly_sensitive,extended_entities,users_mentioned,clean_text,polarity,subjectivity,sentiment,compound
0,False,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 50]",0,1031515975928213505,1031526032153735168,29176998,False,0,1031526032153735168,1031515975928213505,Mon Aug 20 12:59:05 +0000 2018,False,@silentworks @jakuuire @pharingee Nice! I’m ga...,en,silentworks,29176998,NaN,NaN,"[silentworks, jakuuire, pharingee]",nice im game,0.100000,0.700000,positive,0.4215
1,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 16]",0,1031500040408387584,1031503428672008193,29176998,False,0,1031503428672008193,1031500040408387584,Mon Aug 20 11:29:16 +0000 2018,False,@silentworks 💀💀💀,und,silentworks,29176998,NaN,NaN,[silentworks],,0.000000,0.000000,neutral,0.0000
2,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 140]",0,NaN,1031263237025812485,NaN,False,0,1031263237025812485,NaN,Sun Aug 19 19:34:50 +0000 2018,False,RT @IbukunOg: Benjamin @DadaBen_ ! I’m so prou...,en,NaN,NaN,NaN,NaN,"[IbukunOg, DadaBen_]",benjamin im so proud of you i see this guy tak...,0.245455,0.684848,positive,0.6113
3,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 85]",0,NaN,1031262871240486912,NaN,False,0,1031262871240486912,NaN,Sun Aug 19 19:33:23 +0000 2018,False,RT @Mz_Chi: Here is a link to the video if you...,en,NaN,NaN,False,NaN,[Mz_Chi],here is a link to the video if you be interested,0.250000,0.500000,positive,0.4019
4,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 121]",0,NaN,1031262859899154432,NaN,False,0,1031262859899154432,NaN,Sun Aug 19 19:33:20 +0000 2018,False,RT @Mz_Chi: I'm going to make a YouTube video ...,en,NaN,NaN,NaN,NaN,[Mz_Chi],im going to make a youube video showing how yo...,0.000000,0.000000,negative,-0.1280
5,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'user_mentions': [{'name': 'The Chi', 'screen...","[0, 55]",0,NaN,1031262697109835777,NaN,False,0,1031262697109835777,NaN,Sun Aug 19 19:32:41 +0000 2018,False,RT @Mz_Chi: From dusk till dawn https://t.co/B...,en,NaN,NaN,False,{'media': [{'expanded_url': 'https://twitter.c...,[Mz_Chi],from dusk till dawn,0.000000,0.000000,neutral,0.0000
6,True,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 115]",0,NaN,1031205963964067842,NaN,False,0,1031205963964067842,NaN,Sun Aug 19 15:47:15 +0000 2018,False,"RT @allenakinkunle: ""Maybe you don't understan...",ca,NaN,NaN,False,NaN,[allenakinkunle],maybe you dont understand\nẹgbàmí what was he ...,0.000000,0.000000,neutral,0.0000
7,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [], 'symbols': [], 'user_mentions...","[0, 139]",0,NaN,1031147154709069824,NaN,False,0,1031147154709069824,NaN,Sun Aug 19 11:53:34 +0000 2018,False,RT @asemota: There is something that I discove...,en,NaN,NaN,NaN,NaN,[asemota],here is something that i discovered quite earl...,0.152083,0.360417,positive,0.8176
8,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [{'text': 'WISPDEFCONSQUAD', 'ind...","[0, 120]",0,NaN,1031074729610895360,NaN,False,0,1031074729610895360,NaN,Sun Aug 19 07:05:46 +0000 2018,False,RT @Ebunsky: Woohoo! My first LinkedIn article...,en,NaN,NaN,False,NaN,[Ebunsky],woohoo my first linkedin article sharing my de...,0.250000,0.333333,positive,0.7269
9,True,"<a href=""http://twitter.com/download/iphone"" r...","{'hashtags': [{'text': 'forLoopPod', 'indices'..

In [ ]:
# summary_day... positive_count, negative_count, neutral_count

